<a href="https://colab.research.google.com/github/EllaThomasson/labs-ds3001/blob/main/WebScrapingLab_EllaThomasson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1

In [3]:
# I am choosing to look at the appts/housing options on craigslist

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests # Page requests

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0'}
url = 'https://charlottesville.craigslist.org/search/pet#search=1~list~0~0'
raw = requests.get(url,headers=header)


Question 2

In [4]:
from bs4 import BeautifulSoup as soup # HTML parser
bsObj = soup(raw.content,'html.parser') # Parse the html
listings = bsObj.find_all(class_="cl-static-search-result") # Find all listings

In [5]:
import re # Regular expressions

types = ['rabbit', 'yorkie','kitten','bird','snake','macaw','puppies','dog','fish','cat',
          'kittens','cats','rooster','gecko']

data = [] # We'll save our listings in this object
for k in range( len(listings) ):
    title = listings[k].find('div',class_='title').get_text().lower()
    price = listings[k].find('div',class_='price').get_text()
    link = listings[k].find(href=True)['href']
    # Get brand from the title string:
    words = title.split()
    hits = [word for word in words if word in types] # Find brands in the title
    if len(hits) == 0:
        types = 'missing'
    else:
        types = hits[0]
    # Get years from title string:
    regex_search = re.search(r'20[0-9][0-9]|19[0-9][0-9]', title ) # Find year references
    if regex_search is None: # If no hits, record year as missing value
        year = np.nan
    else: # If hits, record year as first match
        year = regex_search.group(0)
    #
    data.append({'title':title,'price':price,'year':year,'link':link,'types':types})

Question 3

In [6]:
data

[{'title': 'large, sturdy rabbit hutches',
  'price': '$0',
  'year': nan,
  'link': 'https://charlottesville.craigslist.org/pet/d/esmont-large-sturdy-rabbit-hutches/7783710794.html',
  'types': 'rabbit'},
 {'title': 'yorkie pups',
  'price': '$0',
  'year': nan,
  'link': 'https://charlottesville.craigslist.org/pet/d/charlottesville-yorkie-pups/7789036918.html',
  'types': 'missing'},
 {'title': 'looking for a kitten',
  'price': '$0',
  'year': nan,
  'link': 'https://charlottesville.craigslist.org/pet/d/charlottesville-looking-for-kitten/7788903625.html',
  'types': 'missing'},
 {'title': 'iso pet bird',
  'price': '$0',
  'year': nan,
  'link': 'https://charlottesville.craigslist.org/pet/d/charlottesville-iso-pet-bird/7788172811.html',
  'types': 'missing'},
 {'title': 'looking for a kitten!',
  'price': '$0',
  'year': nan,
  'link': 'https://charlottesville.craigslist.org/pet/d/charlottesville-looking-for-kitten/7788802039.html',
  'types': 'missing'},
 {'title': 'dog food: rayne

Question 4